In [1]:
%%capture

!pip install gdown gnss_lib_py

In [2]:
%%capture

import gdown

FILE_URL = 'https://drive.google.com/uc?id=1BomwL31p7KvUz-UzQkR4tcB9Paq3IY0Y'
gdown.download(FILE_URL, output='gnss_log_2024_11_28_04_57_56.txt', quiet=True)

In [48]:
import gnss_lib_py as glp

raw_data = glp.AndroidRawGnss(
  input_path="/content/gnss_log_2024_11_28_04_57_56.txt",
  filter_measurements=True,
  measurement_filters={"sv_time_uncertainty" : 500.},
  verbose=True,
)

bias_valid removed 0
bias_uncertainty removed 58
arrival_time removed 0
unknown_constellations removed 0
time_valid removed 0
state_decoded removed 0
sv_time_uncertainty removed 63397
adr_valid removed 379
adr_uncertainty removed 182042


In [49]:
full_states = glp.add_sv_states(raw_data, source="precise", verbose=True)

ephemeris dates needed: [datetime.date(2024, 11, 27), datetime.date(2024, 11, 28)]
using previously downloaded file:
 /content/data/ephemeris/sp3/COD0OPSRAP_20243320000_01D_05M_ORB.SP3
using previously downloaded file:
 /content/data/ephemeris/sp3/COD0OPSRAP_20243330000_01D_05M_ORB.SP3
using previously downloaded file:
 /content/data/ephemeris/sp3/COD0OPSRAP_20243320000_01D_05M_ORB.SP3
using previously downloaded file:
 /content/data/ephemeris/sp3/COD0OPSRAP_20243330000_01D_05M_ORB.SP3
ephemeris dates needed: [datetime.date(2024, 11, 27), datetime.date(2024, 11, 28)]
using previously downloaded file:
 /content/data/ephemeris/clk/COD0OPSRAP_20243320000_01D_30S_CLK.CLK
using previously downloaded file:
 /content/data/ephemeris/clk/COD0OPSRAP_20243330000_01D_30S_CLK.CLK
using previously downloaded file:
 /content/data/ephemeris/clk/COD0OPSRAP_20243320000_01D_30S_CLK.CLK
using previously downloaded file:
 /content/data/ephemeris/clk/COD0OPSRAP_20243330000_01D_30S_CLK.CLK
interpolating sp3 

In [50]:
full_states["corr_pr_m"] = full_states["raw_pr_m"] \
                         + full_states['b_sv_m']

In [51]:
full_states = full_states.where("gnss_id",("gps","galileo"))

In [52]:
wls_estimate = glp.solve_wls(full_states)

/usr/local/lib/python3.10/dist-packages/gnss_lib_py/algorithms/snapshot.py:149: RuntimeWarning:

Need at least four satellites for WLS. Encountered at 44 gps_millis of: 1416779906000, 1416779907000, 1416779925000, 1416779951000, 1416779952000, 1416780043000, 1416780491000, 1416780493000, 1416780494000, 1416780572000, 1416781028000, 1416781071000, 1416781093000, 1416781115000, 1416781116000, 1416781592999, 1416781594999, 1416781604999, 1416781614999, 1416781622999, 1416782117999, 1416782121999, 1416782129999, 1416782588999, 1416782629999, 1416782661999, 1416783064999, 1416783065999, 1416783073999, 1416783075999, 1416783084999, 1416783095999, 1416783103999, 1416783106999, 1416783153999, 1416783188999, 1416783632999, 1416783639999, 1416783743999, 1416783747999, 1416783906999, 1416783909999, 1416783910999, 1416783911999

/usr/local/lib/python3.10/dist-packages/gnss_lib_py/algorithms/snapshot.py:154: RuntimeWarning:

No valid state estimate computed in WLS, returning NaNs.



In [54]:
import numpy as np
wls_estimate = wls_estimate.where('x_rx_wls_m', np.nan, condition='neq')

Empty NavData
Rows: [gps_millis, x_rx_wls_m, y_rx_wls_m, z_rx_wls_m, b_rx_wls_m]
Index: []

In [47]:
raw_fig = glp.plot_map(wls_estimate)
raw_fig.show()